<a href="https://colab.research.google.com/github/hkhong72/big_data/blob/main/Chapter_20_%EC%B9%98%EB%A7%A4%ED%99%98%EC%9E%90_%EB%87%8C_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # KMP에러 방지

In [ ]:
# 데이터 가져오기
!git clone https://github.com/taehojo/data-ch20.git

In [ ]:
# 학습셋 변형
train_datagen = ImageDataGenerator(rescale=1./255, # 주어진 이미지 크기를 설정
                                                  horizontal_flip=True, # 수평 대칭 이미지를 50% 확률로 만들어 추가
                                                  width_shift_range=0.1, # 마찬가지로 위아래로 이동
                                                  height_shift_range=0.1, # 정해진 각도만큼 회전
                                                  #rotation_range=5, # 좌표 하나를 고정시키고 나머지를 이동
                                                  #shear_range=0.7, # 확대 또는 축소
                                                  #zoom_range=[0.9, 2.2], # 수직 대칭 이미지 만들기
                                                  #vertical_flip=True, # 빈 공간 채우는 방법
                                                  #fill_mode='nearest') # nearest 옵션 = 가장 비슷한 색으로 채우기

In [ ]:
train_generator = train_datagen.flow_from_directory('data/train/', # 학습셋 폴더 위치
                                                                    target_size=(150,150),
                                                                    batch_size=5,
                                                                    class_mode='binary')

In [ ]:
# 테스트셋은 이미지 부풀리기 과정을 진행하지 않는다.
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = test_datagen.flow_from_directory('data/test/', # 학습셋 폴더 위치
                                                                    target_size=(150,150),
                                                                    batch_size=5,
                                                                    class_mode='binary')

In [ ]:
# CNN 모델을 만들어 적용해야한다.
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activaiton('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
# 모델 실행 옵션 설정
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])

# 학습 조기 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# 모델 실행
history = model.fit(
        train_generator,
        epochs=100,
        validation_data=test_generator,
        validation_steps=10,
        callbacks=[early_stopping_callback])

In [ ]:
# 검증셋과 학습셋의 오차 저장
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

In [ ]:
# 그래프 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker=' . ', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker=' . ', c="blue", label='Trainset_loss')

# 그래프에 그리드 주고 레이블 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()